In [1]:
print('test')

test


In [71]:
import requests
import pandas as pd
import zipfile
import os
import time
from io import BytesIO
import logging
import sys # Usado para direcionar o log para o console
from datetime import datetime
from tqdm import tqdm
import boto3


# Configura os handlers: um para o arquivo e outro para o console
# file_handler = logging.FileHandler("data_download.log") # Salva em um arquivo
# stream_handler = logging.StreamHandler(sys.stdout) # Mostra no console

In [16]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
logging.basicConfig(level=logging.INFO)
start_date = 2022
end_date = datetime.now().date()
end_year = end_date.year
end_month = end_date.month
output_dir = "/home/jonanov/Documentos/Data_engineer/PROJETOS/ETL_01/data"
os.makedirs(output_dir, exist_ok=True)

# Criar lista de (ano, mês)
periodos = []
for year in range(start_date, end_year + 1):
    for month in range(1, 13):
        if year == end_year and month > end_month:
            break
        periodos.append((year, month))

for year, month in tqdm(periodos, desc="Baixando relatórios", unit="mês"):
    yyyymm = f"{year}{month:02d}"
    url = f"https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{yyyymm}.zip"
    logging.info(f"Processando Relatório_{yyyymm}")

Baixando relatórios:   0%|          | 0/43 [00:00<?, ?mês/s]

2025-07-27 22:45:00,197 - INFO - Processando 202201...
2025-07-27 22:45:00,198 - INFO - Processando 202202...
2025-07-27 22:45:00,198 - INFO - Processando 202203...
2025-07-27 22:45:00,198 - INFO - Processando 202204...
2025-07-27 22:45:00,199 - INFO - Processando 202205...
2025-07-27 22:45:00,199 - INFO - Processando 202206...
2025-07-27 22:45:00,199 - INFO - Processando 202207...
2025-07-27 22:45:00,200 - INFO - Processando 202208...
2025-07-27 22:45:00,200 - INFO - Processando 202209...
2025-07-27 22:45:00,200 - INFO - Processando 202210...
2025-07-27 22:45:00,201 - INFO - Processando 202211...
2025-07-27 22:45:00,201 - INFO - Processando 202212...
2025-07-27 22:45:00,202 - INFO - Processando 202301...
2025-07-27 22:45:00,202 - INFO - Processando 202302...
2025-07-27 22:45:00,202 - INFO - Processando 202303...
2025-07-27 22:45:00,203 - INFO - Processando 202304...
2025-07-27 22:45:00,203 - INFO - Processando 202305...
2025-07-27 22:45:00,203 - INFO - Processando 202306...
2025-07-27

Baixando relatórios: 100%|██████████| 43/43 [00:00<00:00, 2291.10mês/s]


In [ ]:
# for year in range(start_date, end_year + 1):
#     for month in tqdm(range(1, 13), desc="Relatórios"):
#         if year == end_year and month > end_month:
#             break  # não tenta baixar meses futuros
#         time.sleep(1)
#         yyyymm = f"{year}{month:02d}"


Relatórios:  58%|█████▊    | 7/12 [00:07<00:05,  1.00s/it]


In [68]:
class ExtractCvm:

    def __init__(self,start_date, output_dir, end_date=None):
        self.start_date = start_date
        self.output_dir = output_dir
        self.end_date = end_date or datetime.now().date()
        self.end_year = self.end_date.year
        self.end_month = self.end_date.month


    def extract_info_diary(self):

        periodos = []
        for year in range(self.start_date, self.end_year + 1):
            for month in range(1, 13):
                if year == self.end_year and month > self.end_month:
                    break
                periodos.append((year, month))

        for year, month in tqdm(periodos, desc="Baixando relatórios", unit="mês"):
            yyyymm = f"{year}{month:02d}"
            url = f"https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{yyyymm}.zip"
            #logging.info(f"Processando Relatório_{yyyymm}...")
            os.makedirs(self.output_dir, exist_ok=True)

            try:
                response = requests.get(url)
                response.raise_for_status()
                zip_file_in_memory = BytesIO(response.content)

                with zipfile.ZipFile(zip_file_in_memory) as zf:
                    csv_zip = zf.namelist()[0]
                    df = pd.read_csv(zf.open(csv_zip), sep=';', encoding='latin1')
                    output_path = os.path.join(self.output_dir, f"inf_diario_fi_{yyyymm}.csv")
                    df.to_csv(output_path, index=False)

                    logging.info(f"Arquivo salvo em: {output_path}")

            except requests.exceptions.HTTPError as err:
                logging.exception(f"Erro HTTP: O arquivo para {month:02d}/{year} não foi encontrado ou o servidor retornou um erro.")

            except Exception as err:
                logging.exception(f"Erro inesperado em {month:02d}/{year}: {err}")



In [69]:
if __name__ == "__main__":
    import logging
    logging.basicConfig(level=logging.INFO)

    extrator = ExtractCvm(start_date=2022, output_dir="/home/jonanov/Documentos/Data_engineer/PROJETOS/ETL_01/data")
    extrator.extract_info_diary()

Baixando relatórios:   0%|          | 0/43 [00:00<?, ?mês/s]

2025-07-27 23:23:36,121 - INFO - Arquivo salvo em: /home/jonanov/Documentos/Data_engineer/PROJETOS/ETL_01/data/inf_diario_fi_202201.csv


Baixando relatórios:   2%|▏         | 1/43 [00:02<01:56,  2.78s/mês]

2025-07-27 23:23:38,563 - INFO - Arquivo salvo em: /home/jonanov/Documentos/Data_engineer/PROJETOS/ETL_01/data/inf_diario_fi_202202.csv


Baixando relatórios:   5%|▍         | 2/43 [00:05<01:45,  2.58s/mês]

...

Baixando relatórios:  95%|█████████▌| 41/43 [02:06<00:06,  3.20s/mês]

2025-07-27 23:25:43,123 - INFO - Arquivo salvo em: /home/jonanov/Documentos/Data_engineer/PROJETOS/ETL_01/data/inf_diario_fi_202506.csv


Baixando relatórios:  98%|█████████▊| 42/43 [02:09<00:03,  3.18s/mês]

2025-07-27 23:25:45,860 - INFO - Arquivo salvo em: /home/jonanov/Documentos/Data_engineer/PROJETOS/ETL_01/data/inf_diario_fi_202507.csv


Baixando relatórios: 100%|██████████| 43/43 [02:12<00:00,  3.08s/mês]


In [ ]:

from botocore.exceptions import ClientError

class load_bucket:

    def __init__(self):
        self.bucket_name = 'bucket-cvm-fi'
        self.local_path = local_path
        self.s3 = boto3.client(
            "s3",
            endpoint_url="http://localstack:4566",  
            aws_access_key_id="test",            
            aws_secret_access_key="test",
            region_name="us-east-1"
        )

    def create_bucket(self):
        try:
            self.s3.create_bucket(Bucket=self.bucket_name)
            print(f"Bucket '{self.bucket_name}' criado com sucesso.")
        except ClientError as e:
            if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
                print(f"Bucket '{self.bucket_name}' já existe.")
            else:
                print(f"Erro ao criar o bucket: {e}")

        # Lista os buckets
        response = self.s3.list_buckets()
        print("Buckets existentes:")
        for bucket in response.get('Buckets', []):
            print(f" - {bucket['Name']}")
        print('#' * 50)

    def insert_data(self):
        for file_name in os.listdir(self.local_path):
            path_full = os.path.join(self.local_path, file_name)

            if os.path.isfile(path_full):
                chave_s3 = file_name.replace("\\", "/")

                try:
                    with open(path_full, "rb") as f:
                        self.s3.put_object(Bucket=self.bucket_name, Key=chave_s3, Body=f)
                    print(f"Arquivo '{file_name}' enviado com sucesso para '{chave_s3}'.")
                except ClientError as e:
                    print(f"Erro ao enviar o arquivo '{file_name}': {e}")
                except Exception as e:
                    print(f"Erro inesperado ao enviar o arquivo '{file_name}': {e}")
        print('#' * 50)


In [65]:
logging.basicConfig(level=logging.INFO)
start_date = 2022
end_date = datetime.now().date()
end_year = end_date.year
end_month = end_date.month
output_dir = "/home/jonanov/Documentos/Data_engineer/PROJETOS/ETL_01/data"
os.makedirs(output_dir, exist_ok=True)

# Criar lista de (ano, mês)
periodos = []
for year in range(start_date, end_year + 1):
    for month in range(1, 13):
        if year == end_year and month > end_month:
            break
        periodos.append((year, month))

for year, month in tqdm(periodos, desc="Baixando relatórios", unit="mês"):
    yyyymm = f"{year}{month:02d}"
    url = f"https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{yyyymm}.zip"
    logging.info(f"Processando Relatório_{yyyymm}...")

    try:
        response = requests.get(url)
        response.raise_for_status()
        zip_file_in_memory = BytesIO(response.content)

        with zipfile.ZipFile(zip_file_in_memory) as zf:
            csv_zip = zf.namelist()[0]
            df = pd.read_csv(zf.open(csv_zip), sep=';', encoding='latin1')
            output_path = os.path.join(output_dir, f"inf_diario_fi_{yyyymm}.csv")
            df.to_csv(output_path, index=False)

            logging.info(f"Arquivo salvo: {output_path}")

    except requests.exceptions.HTTPError as err:
        logging.exception(f"Erro HTTP: O arquivo para {month:02d}/{year} não foi encontrado ou o servidor retornou um erro.")
    # except requests.exceptions.RequestException as err:
    #     logging.exception(f"Ocorreu um erro de conexão para {month:02d}/{year}: {err}")
    except Exception as err:
        logging.exception(f"Erro inesperado em {month:02d}/{year}: {err}")

Baixando relatórios:   0%|          | 0/43 [00:00<?, ?mês/s]

2025-07-27 22:48:19,382 - INFO - Processando Relatório_202201...
2025-07-27 22:48:22,681 - INFO - Arquivo salvo: /home/jonanov/Documentos/Data_engineer/PROJETOS/ETL_01/data/inf_diario_fi_202201.csv


Baixando relatórios:   2%|▏         | 1/43 [00:03<02:18,  3.30s/mês]

2025-07-27 22:48:22,681 - INFO - Processando Relatório_202202...
2025-07-27 22:48:25,132 - INFO - Arquivo salvo: /home/jonanov/Documentos/Data_engineer/PROJETOS/ETL_01/data/inf_diario_fi_202202.csv


Baixando relatórios:   5%|▍         | 2/43 [00:05<01:54,  2.80s/mês]

...

Baixando relatórios:  95%|█████████▌| 41/43 [02:02<00:06,  3.07s/mês]

2025-07-27 22:50:21,998 - INFO - Processando Relatório_202506...
2025-07-27 22:50:24,987 - INFO - Arquivo salvo: /home/jonanov/Documentos/Data_engineer/PROJETOS/ETL_01/data/inf_diario_fi_202506.csv


Baixando relatórios:  98%|█████████▊| 42/43 [02:05<00:03,  3.05s/mês]

2025-07-27 22:50:24,988 - INFO - Processando Relatório_202507...
2025-07-27 22:50:27,591 - INFO - Arquivo salvo: /home/jonanov/Documentos/Data_engineer/PROJETOS/ETL_01/data/inf_diario_fi_202507.csv


Baixando relatórios: 100%|██████████| 43/43 [02:08<00:00,  2.98s/mês]


In [28]:
df

,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
0,FI,00.017.024/0001-53,2021-07-01,1074549.55,27.512452,1075592.01,0.0,0.00,1
1,FI,00.017.024/0001-53,2021-07-02,1069738.09,27.514565,1075674.60,0.0,0.00,1
2,FI,00.017.024/0001-53,2021-07-05,1069921.24,27.517180,1075776.84,0.0,0.00,1
3,FI,00.017.024/0001-53,2021-07-06,1070097.99,27.519678,1075874.50,0.0,0.00,1
4,FI,00.017.024/0001-53,2021-07-07,1066022.47,27.522013,1071994.92,0.0,3970.88,1
...,...,...,...,...,...,...,...,...,...
469761,FI,97.929.213/0001-34,2021-07-26,71459007.00,9.582287,72818875.37,0.0,0.00,2
469762,FI,97.929.213/0001-34,2021-07-27,70981869.29,9.519419,72341125.87,0.0,0.00,2
469763,FI,97.929.213/0001-34,2021-07-28,72461627.51,9.533863,72450889.67,0.0,0.00,2
469764,FI,97.929.213/0001-34,2021-07-29,72468361.26,9.534696,72457219.46,0.0,0.00,2


In [2]:
import os
import pandas as pd

In [3]:
for i in os.listdir('/home/jonanov/Documentos/Data_engineer/PROJETOS/ETL_01/data'):
    print(i)

inf_diario_fi_202412.csv
inf_diario_fi_202503.csv
inf_diario_fi_202301.csv
inf_diario_fi_202201.csv
inf_diario_fi_202403.csv
inf_diario_fi_202205.csv
inf_diario_fi_202203.csv
inf_diario_fi_202404.csv
inf_diario_fi_202312.csv
inf_diario_fi_202211.csv
inf_diario_fi_202210.csv
inf_diario_fi_202504.csv
inf_diario_fi_202309.csv
inf_diario_fi_202311.csv
inf_diario_fi_202409.csv
inf_diario_fi_202310.csv
inf_diario_fi_202304.csv
inf_diario_fi_202306.csv
inf_diario_fi_202206.csv
inf_diario_fi_202302.csv
inf_diario_fi_202208.csv
inf_diario_fi_202408.csv
inf_diario_fi_202307.csv
inf_diario_fi_202505.csv
inf_diario_fi_202303.csv
inf_diario_fi_202405.csv
inf_diario_fi_202507.csv
inf_diario_fi_202305.csv
inf_diario_fi_202501.csv
inf_diario_fi_202401.csv
inf_diario_fi_202308.csv
inf_diario_fi_202411.csv
inf_diario_fi_202502.csv
inf_diario_fi_202207.csv
inf_diario_fi_202212.csv
inf_diario_fi_202410.csv
inf_diario_fi_202407.csv
inf_diario_fi_202406.csv
inf_diario_fi_202204.csv
inf_diario_fi_202202.csv


In [15]:

for i in os.listdir('/home/jonanov/Documentos/Data_engineer/PROJETOS/ETL_01/data'):
    file_path = os.path.join(f'/home/jonanov/Documentos/Data_engineer/PROJETOS/ETL_01/data',str(i))
    df = pd.read_csv(file_path,encoding='latin1')
    #print(f'DF: {i} : {df.shape[0]} linhas')
    print(f'DF: {i} \n {df.isna().sum()} linhas')
    print('\n #############################################\n')
   

/tmp/ipykernel_366899/299970433.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path,encoding='latin1')


DF: inf_diario_fi_202412.csv 
 TP_FUNDO_CLASSE           0
CNPJ_FUNDO_CLASSE         0
ID_SUBCLASSE         538678
DT_COMPTC                 0
VL_TOTAL                  0
VL_QUOTA                  0
VL_PATRIM_LIQ             0
CAPTC_DIA                 0
RESG_DIA                  0
NR_COTST                  0
dtype: int64 linhas

 #############################################



/tmp/ipykernel_366899/299970433.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path,encoding='latin1')


DF: inf_diario_fi_202503.csv 
 TP_FUNDO_CLASSE           0
CNPJ_FUNDO_CLASSE         0
ID_SUBCLASSE         486624
DT_COMPTC                 0
VL_TOTAL                  0
VL_QUOTA                  0
VL_PATRIM_LIQ             0
CAPTC_DIA                 0
RESG_DIA                  0
NR_COTST                  0
dtype: int64 linhas

 #############################################

DF: inf_diario_fi_202301.csv 
 TP_FUNDO         0
CNPJ_FUNDO       0
DT_COMPTC        0
VL_TOTAL         0
VL_QUOTA         0
VL_PATRIM_LIQ    0
CAPTC_DIA        0
RESG_DIA         0
NR_COTST         0
dtype: int64 linhas

 #############################################

DF: inf_diario_fi_202201.csv 
 TP_FUNDO         0
CNPJ_FUNDO       0
DT_COMPTC        0
VL_TOTAL         0
VL_QUOTA         0
VL_PATRIM_LIQ    0
CAPTC_DIA        0
RESG_DIA         0
NR_COTST         0
dtype: int64 linhas

 #############################################

DF: inf_diario_fi_202403.csv 
 TP_FUNDO_CLASSE           0
CNPJ_FUNDO_CLASSE  

/tmp/ipykernel_366899/299970433.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path,encoding='latin1')


...

DF: inf_diario_fi_202411.csv 
 TP_FUNDO_CLASSE           0
CNPJ_FUNDO_CLASSE         0
ID_SUBCLASSE         486254
DT_COMPTC                 0
VL_TOTAL                  0
VL_QUOTA                  0
VL_PATRIM_LIQ             0
CAPTC_DIA                 0
RESG_DIA                  0
NR_COTST                  0
dtype: int64 linhas

 #############################################



/tmp/ipykernel_366899/299970433.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path,encoding='latin1')


DF: inf_diario_fi_202502.csv 
 TP_FUNDO_CLASSE           0
CNPJ_FUNDO_CLASSE         0
ID_SUBCLASSE         510948
DT_COMPTC                 0
VL_TOTAL                  0
VL_QUOTA                  0
VL_PATRIM_LIQ             0
CAPTC_DIA                 0
RESG_DIA                  0
NR_COTST                  0
dtype: int64 linhas

 #############################################

DF: inf_diario_fi_202207.csv 
 TP_FUNDO         0
CNPJ_FUNDO       0
DT_COMPTC        0
VL_TOTAL         0
VL_QUOTA         0
VL_PATRIM_LIQ    0
CAPTC_DIA        0
RESG_DIA         0
NR_COTST         0
dtype: int64 linhas

 #############################################

DF: inf_diario_fi_202212.csv 
 TP_FUNDO         0
CNPJ_FUNDO       0
DT_COMPTC        0
VL_TOTAL         0
VL_QUOTA         0
VL_PATRIM_LIQ    0
CAPTC_DIA        0
RESG_DIA         0
NR_COTST         0
dtype: int64 linhas

 #############################################



/tmp/ipykernel_366899/299970433.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path,encoding='latin1')


DF: inf_diario_fi_202410.csv 
 TP_FUNDO_CLASSE           0
CNPJ_FUNDO_CLASSE         0
ID_SUBCLASSE         592988
DT_COMPTC                 0
VL_TOTAL                  0
VL_QUOTA                  0
VL_PATRIM_LIQ             0
CAPTC_DIA                 0
RESG_DIA                  0
NR_COTST                  0
dtype: int64 linhas

 #############################################

DF: inf_diario_fi_202407.csv 
 TP_FUNDO_CLASSE           0
CNPJ_FUNDO_CLASSE         0
ID_SUBCLASSE         593746
DT_COMPTC                 0
VL_TOTAL                  0
VL_QUOTA                  0
VL_PATRIM_LIQ             0
CAPTC_DIA                 0
RESG_DIA                  0
NR_COTST                  0
dtype: int64 linhas

 #############################################

DF: inf_diario_fi_202406.csv 
 TP_FUNDO_CLASSE           0
CNPJ_FUNDO_CLASSE         0
ID_SUBCLASSE         517137
DT_COMPTC                 0
VL_TOTAL                  0
VL_QUOTA                  0
VL_PATRIM_LIQ             0
CAPTC_DIA    

In [13]:
sum(lista_linhas_df)

22289942

In [8]:
df.isna().sum()

TP_FUNDO         0
CNPJ_FUNDO       0
DT_COMPTC        0
VL_TOTAL         0
VL_QUOTA         0
VL_PATRIM_LIQ    0
CAPTC_DIA        0
RESG_DIA         0
NR_COTST         0
dtype: int64